In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pic

from sklearn.model_selection import train_test_split

from templates import classifications_per_trace, reconstruct_key, guessing_entropy

In [2]:
import deep_learning

2024-05-28 10:33:06.421887: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-28 10:33:06.453176: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 10:33:07.308763: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
with open("full_data.pic", "rb") as r:
    seeds_total, traces_total, key, round_perms_labels, copy_perms_labels, masks_labels, rws_perms_labels, rws_masks_labels = pic.load(r)

In [4]:
X_total, y_total = deep_learning.prepare_data_dl(True, traces_total, round_perms_labels, copy_perms_labels, masks_labels, rws_perms_labels, rws_masks_labels)

X_profiling, X_extraction = train_test_split(X_total, test_size=100_000, random_state=0)
y_profiling = {}
y_extraction = {}
for label in y_total.keys():
    train, test = train_test_split(y_total[label], test_size=100_000, random_state=0)
    y_profiling[label] = train
    y_extraction[label] = test

In [6]:
y_predicted = deep_learning.extract_key(X_extraction, "./resnet_models/resnet_rws_300000.keras")
y_predicted = [np.log(pred) for pred in y_predicted]

2024-05-28 10:34:48.027302: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:41:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-28 10:34:48.028215: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-28 10:34:48.347070: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:41:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-28 10:34:48.347120: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-28 10:34:48.347144: I external/local_xla/xla/stream_executor

  14/3125 ━━━━━━━━━━━━━━━━━━━━ 38s 12ms/step

I0000 00:00:1716885384.935117    4246 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 41s 11ms/step


In [7]:
with open("y_extraction_rws_300000.pic", "wb") as w:
    pic.dump(y_predicted, w)

In [14]:
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical

for i, k in enumerate(y_extraction.keys()):
    print(f"{k}: {np.count_nonzero(np.argmax(y_predicted[i], axis=1) == y_extraction[k])}/{y_extraction[k].shape[0]} = {np.count_nonzero(np.argmax(y_predicted[i], axis=1) == y_extraction[k]) / y_extraction[k].shape[0]}")

np.mean([categorical_crossentropy(to_categorical(y_extraction[k]), y_predicted[i]) for i, k in enumerate(y_extraction.keys())], axis=1)

rws_perm: 79300/100000 = 0.793
rws_mask_0_0: 10114/100000 = 0.10114
rws_mask_0_1: 7932/100000 = 0.07932
rws_mask_1_0: 10294/100000 = 0.10294
rws_mask_1_1: 8784/100000 = 0.08784
rws_mask_2_0: 10086/100000 = 0.10086
rws_mask_2_1: 8502/100000 = 0.08502
rws_mask_3_0: 10238/100000 = 0.10238
rws_mask_3_1: 8377/100000 = 0.08377
rws_mask_4_0: 10004/100000 = 0.10004
rws_mask_4_1: 8645/100000 = 0.08645
rws_mask_5_0: 9979/100000 = 0.09979
rws_mask_5_1: 8626/100000 = 0.08626
rws_mask_6_0: 10267/100000 = 0.10267
rws_mask_6_1: 8652/100000 = 0.08652
rws_mask_7_0: 9560/100000 = 0.0956
rws_mask_7_1: 8535/100000 = 0.08535
rws_mask_8_0: 10053/100000 = 0.10053
rws_mask_8_1: 8553/100000 = 0.08553
rws_mask_9_0: 10250/100000 = 0.1025
rws_mask_9_1: 8601/100000 = 0.08601
rws_mask_10_0: 10212/100000 = 0.10212
rws_mask_10_1: 8495/100000 = 0.08495
rws_mask_11_0: 9770/100000 = 0.0977
rws_mask_11_1: 8134/100000 = 0.08134
rws_mask_12_0: 9689/100000 = 0.09689
rws_mask_12_1: 8642/100000 = 0.08642
rws_mask_13_0: 9572/1

array([11.175485 ,  2.8469002,  2.8105667,  2.8499033,  2.8048055,
        2.84496  ,  2.8059413,  2.8478436,  2.808064 ,  2.8474884,
        2.8064747,  2.8453805,  2.8055534,  2.8459888,  2.8069685,
        2.8455784,  2.8063443,  2.8460186,  2.8068824,  2.8483372,
        2.8063397,  2.8447137,  2.8051   ,  2.847312 ,  2.8080566,
        2.8452256,  2.8041363,  2.8405175,  2.806696 ,  2.846242 ,
        2.807208 ,  2.8457706,  2.8037035,  2.844479 ,  2.8089075,
        2.8493125,  2.8069673,  2.8453825,  2.8073444,  2.844664 ,
        2.8085868,  2.8492248,  2.8084822,  2.8424695,  2.8075347,
        2.8474298,  2.8074918,  2.8477938,  2.80546  ], dtype=float32)